In [54]:
import os
import pandas as pd
import numpy as np
import pickle
import collections
from Aggregation import Aggregate
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.width', 1500)

In [55]:
class Standardize:

    def __init__(self, sym):
        self.symbol = sym
        self.statements = Aggregate(sym).statements

        self.balance = self.statements[0] / 10**6
        self.income = self.statements[1] / 10**6
        self.cash = self.statements[2] / 10**6
        
        self.fcfe = self.free_cash_flow_equity()


    def standard_balance(self):
        """
         _____________
        | Assets:           Liabilities
         -------------       Current:
        | Current:
        |
        |   Cash                A/P
        |   A/R                 CPLTD
        |   Inventory           Other
        |   Other
        |
        | Non-Current:       Non-Current:
        |
        |   PP&E                Notes Payable
        |   Other               Other



        """

        self.balance.index[self.balance.index.str.contains('Current')].tolist()

        return 0


    def standard_income(self):
        return 0

    def standard_cash(self):
        return 0

    def change_nwc(self):
        """
        Calculate Change in Net Working Capital. There are two main ways to do this:

            1. From The Statement of Cash Flows
            2. From The Balance Sheet
        """


        net = self.balance.loc['AssetsCurrent'] - self.balance.loc['LiabilitiesCurrent']
        self.chg_wc = net - net.shift(1)

        return self.chg_wc.fillna(0)

    def CapEx(self):
        self.capex = self.cash.loc[self.cash.index[self.cash.index.str.contains('PropertyPlantAndE')]]
        
        if self.capex.empty:
            return 0
        else:
            self.capex = self.capex.loc[self.capex.index.values[0]]
            return self.capex.fillna(0)

    def debt_cash(self):
        issued = self.cash.loc[self.cash.index[(self.cash.index.str.contains('Debt')) & (self.cash.index.str.contains('Iss'))]]
        if not issued.empty:
            issued = issued.loc[issued.index[0]].fillna(0)
        
        payments = self.cash.loc[self.cash.index[(self.cash.index.str.contains('Debt')) & (self.cash.index.str.contains('Repay'))]]
        if not payments.empty:
            payments = payments.loc[payments.index[0]].fillna(0)
            
        if payments.empty:
            return issued

        elif issued.empty:
            return payments

        else:
            return (issued - payments).fillna(0)

    def DeprAmort(self):
        DA = self.cash.loc[self.cash.index[(self.cash.index.str.contains('Depre')) | (self.cash.index.str.contains('Amort'))]]
        if not DA.empty:
            
            return DA.loc[DA.index.values[0]].fillna(0)
        else:
            return 0
        
    def interest_expense(self):
        int_exp = self.income.loc['InterestExpense']
        if not int_exp.empty:
            return int_exp.fillna(0)
        else:
            return 0
        
        
        
    def free_cash_flow_equity(self):
        """
        FCFe = NI + D/A - ∆ nwc - CapEx  + Net Debt
        """
        
        
        try:
            NI = self.income.loc['NetIncomeLoss'].fillna(0)
        except:
            NI = self.income.loc[self.income.index[self.income.index.str.contains('NetIncome')]].fillna(0)
            if NI.empty:
                NI = self.income.loc[self.income.index[self.income.index.str.contains('OperatingIncome')]].fillna(0)
                NI = NI.loc[NI.index.values[0]]
        
        
        DA = self.DeprAmort()
        nwc_chg = self.change_nwc()
        capex = self.CapEx()
        
        net_debt = self.debt_cash()
        if net_debt.empty:
            net_debt = 0
        
        NIDA = ((NI + DA).fillna(0) + net_debt).fillna(0)
        nwcap = (nwc_chg + capex).fillna(0)
        return (NIDA - nwcap)
    
    
    def free_cash_flow_firm(self):
        """
        FCFf = EBIT + Interest Expense + D/A - ∆ nwc - CapEx
        """
        
        
        try:
            EBIT = self.income.loc['OperatingIncomeLoss'].fillna(0)
        except:
            EBIT = self.income.loc[self.income.index[self.income.index.str.contains('OperatingIncome')]].fillna(0)
            if EBIT.empty:
                EBIT = self.income.loc[self.income.index[self.income.index.str.contains('OperatingIncome')]].fillna(0)
                EBIT = EBIT.loc[EBIT.index.values[0]]
        
        
        DA = self.DeprAmort()
        nwc_chg = self.change_nwc()
        capex = self.CapEx()
        
        int_exp = self.interest_expense()
        
        EBITDA = ((EBIT + DA).fillna(0) + int_exp).fillna(0)
        nwcap = (nwc_chg + capex).fillna(0)
        return (EBITDA - nwcap)


In [56]:
tsla = Standardize('TSLA')
tsla.free_cash_flow_equity()
tsla.free_cash_flow_firm()

2009     -42.426
2010    -175.426
2011    -463.600
2012    -365.204
2013    -595.770
2014   -1324.469
2015    -693.841
dtype: float64

In [49]:
amzn.income.loc['InterestExpense']

2007     77.0
2008     71.0
2009     34.0
2010     39.0
2011     65.0
2012     92.0
2013    141.0
2014    210.0
2015    459.0
Name: InterestExpense, dtype: float64

In [12]:
(nvda.income.loc['Revenues'].pct_change() * 100).mean()
# (nvda.income.loc['NetIncomeLoss'].pct_change() * 100)
nvda.income

,2008,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,,
CostOfRevenue,2228.580,2250.590,2149.522,2134.219,1941.413,2053.816,1862.399,2082.030,2199.0
GrossProfit,1869.280,1174.269,1176.923,1409.090,2056.517,2226.343,2267.763,2599.477,2811.0
IncomeTaxExpenseBenefit,103.696,-12.913,-14.307,18.023,82.306,99.503,70.264,124.249,129.0
InterestExpense,0.054,0.406,3.320,3.127,3.089,3.294,10.443,46.133,47.0
InvestmentIncomeInterest,64.289,42.859,23.115,19.057,19.149,19.908,17.119,28.090,39.0
NetIncomeLoss,797.645,-30.041,-67.987,253.146,581.090,562.536,439.990,630.587,614.0
OperatingExpenses,1032.934,1244.969,1275.868,1153.343,1408.218,1578.104,1771.536,1840.488,2064.0
OperatingIncomeLoss,836.346,-70.700,-98.945,255.747,648.299,648.239,496.227,758.989,747.0
OtherNonoperatingIncomeExpense,0.760,-14.707,-3.144,-0.508,-0.963,-2.814,7.351,13.890,4.0


In [22]:
((nvda.income.pct_change(axis=1) * 100).dropna(axis=1)).applymap(lambda x: '%.2f' % x)

,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,
CostOfRevenue,0.99,-4.49,-0.71,-9.03,5.79,-9.32,11.79,5.62
GrossProfit,-37.18,0.23,19.73,45.95,8.26,1.86,14.63,8.14
IncomeTaxExpenseBenefit,-112.45,10.80,-225.97,356.67,20.89,-29.39,76.83,3.82
InterestExpense,651.85,717.73,-5.81,-1.22,6.64,217.03,341.76,1.88
InvestmentIncomeInterest,-33.33,-46.07,-17.56,0.48,3.96,-14.01,64.09,38.84
NetIncomeLoss,-103.77,126.31,-472.34,129.55,-3.19,-21.78,43.32,-2.63
OperatingExpenses,20.53,2.48,-9.60,22.10,12.06,12.26,3.89,12.14
OperatingIncomeLoss,-108.45,39.95,-358.47,153.49,-0.01,-23.45,52.95,-1.58
OtherNonoperatingIncomeExpense,-2035.13,-78.62,-83.84,89.57,192.21,-361.23,88.95,-71.20


In [28]:
income = nvda.income.loc[['Revenues', 'CostOfRevenue', 'GrossProfit', 'ResearchAndDevelopmentExpense', 'SellingGeneralAndAdministrativeExpense', 
                'OperatingExpenses', 'OperatingIncomeLoss', 'InterestExpense', 'IncomeTaxExpenseBenefit', 
                'InvestmentIncomeInterest', 'OtherNonoperatingIncomeExpense', 'NetIncomeLoss']]
income

,2008,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,,
Revenues,4097.860,3424.859,3326.445,3543.309,3997.930,4280.159,4130.162,4681.507,5010.0
CostOfRevenue,2228.580,2250.590,2149.522,2134.219,1941.413,2053.816,1862.399,2082.030,2199.0
GrossProfit,1869.280,1174.269,1176.923,1409.090,2056.517,2226.343,2267.763,2599.477,2811.0
ResearchAndDevelopmentExpense,691.637,855.879,908.851,848.830,1002.605,1147.282,1335.834,1359.725,1331.0
SellingGeneralAndAdministrativeExpense,341.297,362.222,367.017,361.513,405.613,430.822,435.702,480.763,602.0
OperatingExpenses,1032.934,1244.969,1275.868,1153.343,1408.218,1578.104,1771.536,1840.488,2064.0
OperatingIncomeLoss,836.346,-70.700,-98.945,255.747,648.299,648.239,496.227,758.989,747.0
InterestExpense,0.054,0.406,3.320,3.127,3.089,3.294,10.443,46.133,47.0
IncomeTaxExpenseBenefit,103.696,-12.913,-14.307,18.023,82.306,99.503,70.264,124.249,129.0


In [29]:
((income.pct_change(axis=1) * 100).dropna(axis=1)).applymap(lambda x: '%.2f' % x)

,2009,2010,2011,2012,2013,2014,2015,2016
Fiel,,,,,,,,
Revenues,-16.42,-2.87,6.52,12.83,7.06,-3.50,13.35,7.02
CostOfRevenue,0.99,-4.49,-0.71,-9.03,5.79,-9.32,11.79,5.62
GrossProfit,-37.18,0.23,19.73,45.95,8.26,1.86,14.63,8.14
ResearchAndDevelopmentExpense,23.75,6.19,-6.60,18.12,14.43,16.43,1.79,-2.11
SellingGeneralAndAdministrativeExpense,6.13,1.32,-1.50,12.20,6.22,1.13,10.34,25.22
OperatingExpenses,20.53,2.48,-9.60,22.10,12.06,12.26,3.89,12.14
OperatingIncomeLoss,-108.45,39.95,-358.47,153.49,-0.01,-23.45,52.95,-1.58
InterestExpense,651.85,717.73,-5.81,-1.22,6.64,217.03,341.76,1.88
IncomeTaxExpenseBenefit,-112.45,10.80,-225.97,356.67,20.89,-29.39,76.83,3.82


In [33]:
nvda.income.iloc[:, -1].sort_values(ascending=False)

Fiel
Revenues                                  5010.0
GrossProfit                               2811.0
CostOfRevenue                             2199.0
OperatingExpenses                         2064.0
ResearchAndDevelopmentExpense             1331.0
OperatingIncomeLoss                        747.0
NetIncomeLoss                              614.0
SellingGeneralAndAdministrativeExpense     602.0
IncomeTaxExpenseBenefit                    129.0
InterestExpense                             47.0
InvestmentIncomeInterest                    39.0
OtherNonoperatingIncomeExpense               4.0
Name: 2016, dtype: float64